## Linear Regression

In [66]:
# Get user input for the week number to predict
num_week = int(input("Enter the week to predict: "))

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce

positions = ["qb", "rb", "wr", "te"]
positions = ["qb"]


for pos in positions:
    # Load your dataset
    data = pd.read_csv("datasets/weekly_scoring.csv")

    # Preprocessing
    data = data[data['POS'] == pos]
    weights = data['WEIGHT']
    
    if(pos == 'qb'):
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING LG', 'RECEIVING 20+', 'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        'RUSHING 20+', 'DATE', 'WEIGHT', 'YEAR'])
        # Define the list of variables to predict
        var_list = ['PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'WEEK', 'AVG_FPTS', 'MAX_FPTS', 'MIN_FPTS', 'VAR_FPTS']
    if(pos == 'rb'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT', 'RECEIVING LG', 'RECEIVING 20+',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RUSHING Y/A', 'RUSHING LG',
        'RUSHING 20+', 'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']
    if(pos == 'wr'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']
    if(pos == 'te'):
        # Define the list of variables to drop and predict
        data = data.drop(columns=['POS RANK', 'POS', 'MISC G', 'MISC ROST', 'MISC FPTS/G', 'PASSING CMP', 'PASSING ATT', 'PASSING PCT', 'PASSING YDS', 
        'PASSING Y/A', 'PASSING TD', 'PASSING INT',
        'PASSING SACKS', 'YEAR', 'WEIGHT', 'DATE', 'RUSHING Y/A', 'RUSHING LG', 'RUSHING 20+'])
        var_list = ['RECEIVING REC', 'RECEIVING TGT', 'RECEIVING YDS', 'RECEIVING Y/R',
        'RECEIVING TD', 'RECEIVING LG', 'RECEIVING 20+',
        'RUSHING ATT', 'RUSHING YDS', 'RUSHING TD', 'MISC FL', 'MISC FPTS', 'WEEK']

# Without Player
    # # Extracting features and target variable
    # X = data[var_list]  # Ensure 'MISC FPTS' is not in var_list
    # y = data['MISC FPTS']

    # # Splitting the dataset into training and testing sets
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # # Creating and training the linear regression model
    # model = LinearRegression()
    # model.fit(X_train, y_train)

    # # Predicting on the test set
    # y_pred = model.predict(X_test)

    # # Evaluating the model (optional)
    # mse = mean_squared_error(y_test, y_pred)
    # print(f'Mean Squared Error: {mse}')

    # # Printing the coefficients
    # coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})
    # print(coefficients)

# With Player
    # Extracting features and target variable
    X = data[var_list]  # Ensure 'MISC FPTS' is not in var_list
    X['PLAYER'] = data['PLAYER']  # Include 'Player' as a feature

    # Handling missing values in numeric columns using SimpleImputer
    numeric_cols = X.select_dtypes(include='number').columns
    imputer = SimpleImputer(strategy='mean')  # You can change the strategy as needed
    X[numeric_cols] = imputer.fit_transform(X[numeric_cols])

    # Target encoding 'Player' variable --> replaces the Player value with the mean FPTS for each player
    encoder = ce.TargetEncoder(cols=['PLAYER'])
    X = encoder.fit_transform(X, y)

    # Convert categorical variable 'Player' to one-hot encoding
    # X = pd.get_dummies(X, columns=['PLAYER'], drop_first=True)

    y = data['MISC FPTS']

    # Splitting the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creating and training the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predicting on the test set
    y_pred = model.predict(X_test)

    # Evaluating the model (optional)
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse}')

    # Printing the coefficients
    coefficients = pd.DataFrame({'Variable': X.columns, 'Coefficient': model.coef_})
    print(coefficients)

    # Predicting on the entire dataset
    data['Predicted_FPTS'] = model.predict(X)

    # Organizing the results into a new DataFrame
    result_df = data[['PLAYER', 'Predicted_FPTS']].drop_duplicates()
    result_df = result_df.sort_values(by='Predicted FPTS', ascending=False)

    # Displaying the result DataFrame
    result_df.to_csv("results.csv", index=False)


    # # Create a list of dictionaries to store the results
    # results_list = []

    # for player in unique_players:
    #     # Create a DataFrame with all zeros
    #     week6_data = pd.DataFrame(0, index=range(1), columns=X.columns)
    #     # Set the corresponding player's column to 1 for prediction
    #     week6_data[player] = 1
    #     # Make a prediction for the player
    #     misc_fpts_prediction = lr_model.predict(week6_data)
    #     results_list.append({'Player': player, 'MISC FPTS': misc_fpts_prediction[0]})

    # # Convert the list of dictionaries into a DataFrame
    # results_df = pd.DataFrame(results_list)

    # results_df = results_df.sort_values(by='MISC FPTS', ascending=False)

    # # # Save the results to a CSV file
    # file_name = f"predictions/LRweek{num_week}{pos}unweighted.csv"
    # results_df.to_csv(file_name, index=False)
    


Mean Squared Error: 0.20118776911889436
         Variable  Coefficient
0     PASSING CMP    -0.014646
1     PASSING ATT     0.014909
2     PASSING PCT    -0.000509
3     PASSING YDS     0.040418
4     PASSING Y/A    -0.008314
5      PASSING TD     4.028847
6     PASSING INT    -1.018088
7   PASSING SACKS    -0.008078
8     RUSHING ATT    -0.002888
9     RUSHING YDS     0.101835
10     RUSHING TD     6.078470
11        MISC FL    -1.966953
12           WEEK    -0.001122
13       AVG_FPTS    -0.000677
14       MAX_FPTS    -0.002346
15       MIN_FPTS    -0.017334
16       VAR_FPTS     0.005373
17         PLAYER     0.002047


/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_15125/3813703442.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['PLAYER'] = data['PLAYER']  # Include 'Player' as a feature
/var/folders/qd/rhs7cnfj66l4p8v_yrkw8n4m0000gn/T/ipykernel_15125/3813703442.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numeric_cols] = imputer.fit_transform(X[numeric_cols])


KeyError: 'Predicted FPTS'